# Actividad Guiada 2 de Algoritmos de Optimizacion

Nombre: Roberto Saul Cova Rocamora <br>
[enlace google colab] <br>
[enlace github]

In [26]:
import math
import random

## Programacion Dinamica
### Problema 1: Viaje por el rio

In [16]:
# Viaje por el rio

TARIFAS = [
[0,5,4,3,float("inf"),float("inf"),float("inf")],   #desde nodo 0
[float("inf"),0,float("inf"),2,3,float("inf"),11],  #desde nodo 1
[float("inf"),float("inf"), 0,1,float("inf"),4,10], #desde nodo 2
[float("inf"),float("inf"),float("inf"), 0,5,6,9],
[float("inf"),float("inf"), float("inf"),float("inf"),0,float("inf"),4],
[float("inf"),float("inf"), float("inf"),float("inf"),float("inf"),0,3],
[float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),float("inf"),0]
]

TARIFAS

[[0, 5, 4, 3, inf, inf, inf],
 [inf, 0, inf, 2, 3, inf, 11],
 [inf, inf, 0, 1, inf, 4, 10],
 [inf, inf, inf, 0, 5, 6, 9],
 [inf, inf, inf, inf, 0, inf, 4],
 [inf, inf, inf, inf, inf, 0, 3],
 [inf, inf, inf, inf, inf, inf, 0]]

In [17]:
def Precios(TARIFAS):
################################################################ 
  #Total de Nodos
  N = len(TARIFAS[0])
  
  #Inicialización de la tabla de precios
  PRECIOS = [ [float("inf")]*N for i in [float("inf")]*N]  #n x n 
  RUTA = [ [""]*N for i in [""]*N]
  
  #Se recorren todos los nodos con dos bucles(origen - destino)
  #  para ir construyendo la matriz de PRECIOS
  for i in range(N-1):
    for j in range(i+1, N):
      MIN = TARIFAS[i][j]
      RUTA[i][j] = i
      
      for k in range(i, j):
        if PRECIOS[i][k] + TARIFAS[k][j] < MIN:
            MIN = min(MIN, PRECIOS[i][k] + TARIFAS[k][j] )
            RUTA[i][j] = k
        PRECIOS[i][j] = MIN
        
  return PRECIOS,RUTA


In [18]:
PRECIOS,RUTA = Precios(TARIFAS)  
#print(PRECIOS[0][6])

print("PRECIOS")
for i in range(len(TARIFAS)):
  print(PRECIOS[i])

print("\nRUTA")  
for i in range(len(TARIFAS)):
  print(RUTA[i])

PRECIOS
[inf, 5, 4, 3, 8, 8, 11]
[inf, inf, inf, 2, 3, 8, 7]
[inf, inf, inf, 1, 6, 4, 7]
[inf, inf, inf, inf, 5, 6, 9]
[inf, inf, inf, inf, inf, inf, 4]
[inf, inf, inf, inf, inf, inf, 3]
[inf, inf, inf, inf, inf, inf, inf]

RUTA
['', 0, 0, 0, 1, 2, 5]
['', '', 1, 1, 1, 3, 4]
['', '', '', 2, 3, 2, 5]
['', '', '', '', 3, 3, 3]
['', '', '', '', '', 4, 4]
['', '', '', '', '', '', 5]
['', '', '', '', '', '', '']


In [19]:
#Calculo de la ruta usando la matriz RUTA
def calcular_ruta(RUTA, desde, hasta):
  if desde == RUTA[desde][hasta]:
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(RUTA, 0,6)


La ruta es:


'0,2,5'

## Ramificacion y Poda
### Problema 2: Asignacion de tareas

El numero de posibles soluciones es n!

In [20]:
#Asignacion de tareas - Ramificación y Poda
################################################################################
#    T A R E A 
#   A
#   G
#   E
#   N
#   T
#   E

COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [21]:
#Calculo del valor de una solucion parcial
def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR += COSTES[S[i]][i] 
  return VALOR



valor((0,),COSTES)

11

In [22]:
#Coste inferior para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range( len(S), len(COSTES)   ):
    VALOR += min( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range( len(S), len(COSTES)   ):
    VALOR += max( [ COSTES[j][i] for j in range(len(S), len(COSTES))  ])
  return VALOR

CI((0,1),COSTES)

68

In [23]:
#Genera tantos hijos como como posibilidades haya para la siguiente elemento de la tupla
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N ):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)    })
  return HIJOS

In [24]:
crear_hijos((0,) , 4)

[{'s': (0, 1)}, {'s': (0, 2)}, {'s': (0, 3)}]

In [25]:
def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(COSTES)
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple( i for i in range(len(COSTES)) )
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  #print("Cota Superior:", CotaSup)

  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((),COSTES)    } )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [ min(NODOS, key=lambda x:x['ci']) ][0]['s']
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION) ]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION  ]
    if len(NODO_FINAL ) >0: 
      #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION  ] )
      if NODO_FINAL[0]['ci'] < CotaSup:
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL
 
    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup   ]

    #Añadimos los hijos 
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor    ]
   
  print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION  )
      


ramificacion_y_poda(COSTES)

La solucion final es: [{'s': (1, 2, 0, 3), 'ci': 64}]  en  10  iteraciones  para dimension:  4


In [49]:
import itertools

def fuerza_bruta(COSTES):
    dimension = len(COSTES)
    mejor_valor = float('inf')
    mejor_solucion = ()
    iteracion = 0
    for s in list(itertools.permutations(range(len(COSTES)))):
        iteracion +=1
        valor_temp = valor(s,COSTES)
        if valor_temp < mejor_valor:
            mejor_valor = valor_temp
            mejor_solucion = s
            
    print("La solucion final es:" ,{'s':mejor_solucion, 'ci':mejor_valor} , " en " , iteracion , " iteraciones" , " para dimension: " ,dimension  )

La solucion final es: {'s': (3, 2, 1, 0), 'ci': 560}  en  24  iteraciones  para dimension:  4


In [81]:
random.seed(42)
N = 16
COSTES =  [[random.randint(0,1000) for j in range(N)] for i in range(N)]


In [67]:
%time fuerza_bruta(COSTES)

La solucion final es: {'s': (9, 10, 2, 6, 7, 3, 5, 1, 4, 0, 8), 'ci': 1176}  en  39916800  iteraciones  para dimension:  11
CPU times: total: 42.6 s
Wall time: 44.6 s


A partir de la dimension 11, no se puede utilizar fuerza bruta. Ya que mientras que con dimension 10 tarda ~3s, mientras que con 11 tarda ~45s

In [75]:
%time ramificacion_y_poda(COSTES)

La solucion final es: [{'s': (9, 6, 8, 2, 4, 1, 10, 7, 5, 0, 3, 11), 'ci': 1898}]  en  20991  iteraciones  para dimension:  12
CPU times: total: 31.6 s
Wall time: 34.2 s


Depende de la tabla propuesta, ramificacion tarda más o menos en conseguir la solución, dependiendo de cuantas soluciones tiene que explorar. Como vemos, para dimension 12 y esa matriz, tarda ~35s. En cambio, para 13 dimensiones tarde ~0.3s

Para dimension 16 tarda más de 3 minutos, pero no se puede descartar, que haya otra matriz de dimension mayor, para la que pueda dar un resultado en buen tiempo.

In [78]:
%time ramificacion_y_poda(COSTES)

La solucion final es: [{'s': (1, 5, 12, 2, 9, 4, 3, 0, 7, 6, 8, 10, 11), 'ci': 1742}]  en  1847  iteraciones  para dimension:  13
CPU times: total: 172 ms
Wall time: 279 ms


In [82]:
%time ramificacion_y_poda(COSTES)

KeyboardInterrupt: 